In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
rent_csv_file = './data/renttherunway_final_data.csv' 
# mete_csv_file = 'modcloth_final_data.csv' 

rent_data = pd.read_csv(rent_csv_file)
# mete_data = pd.read_csv(mete_csv_file)

In [3]:
def height_to_cm(height):
    if not isinstance(height, str): 
        return None 
    match = re.match(r"(\d+)' (\d+)\"", height)
    if match:
        feet = int(match.group(1)) 
        inches = int(match.group(2)) 
        return feet * 30.48 + inches * 2.54
    else:
        return None

convert height, weight. Using mean fill nan.

In [4]:
rent_data['height_cm'] = rent_data['height'].apply(height_to_cm)
rent_data['weight_kg'] = rent_data['weight'].str.replace('lbs', '').astype(float) * 0.453592

In [5]:
columns_to_fill = ['height_cm', 'weight_kg']  
for col in columns_to_fill:
    mean_value = rent_data[col].mean()  
    rent_data[col] = rent_data[col].fillna(mean_value)  

change clothing size by sort out extreme value, using 98% of the clothing size

In [6]:
threshold = rent_data['size'].quantile(0.98)
df1 = rent_data[rent_data['size'] <= threshold]

In [7]:
# def plot_combined_distributions(data_full, data_filtered):
#     fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

#     # Full data distribution
#     sns.countplot(ax=axes[0], x='size', data=data_full, order=sorted(data_full['size'].unique()))
#     axes[0].set_xlabel('Clothing Size')
#     axes[0].set_ylabel('Count')
#     axes[0].set_title('Full Distribution of Clothing Size')
#     axes[0].axvline(x=np.percentile(data_full['size'], 98), color='red', linestyle='--', label='98th Percentile')
#     axes[0].legend()
#     axes[0].tick_params(axis='x', rotation=45)

#     # Filtered data distribution
#     sns.countplot(ax=axes[1], x='size', data=data_filtered, order=sorted(data_filtered['size'].unique()))
#     axes[1].set_xlabel('Clothing Size')
#     axes[1].set_title('Filtered Distribution of Clothing Size (<= 98th Percentile)')
#     axes[1].tick_params(axis='x', rotation=45)
    
    
#     # Adjust layout
#     plt.tight_layout()
#     plt.savefig('Clothing_size.png')
    
#     plt.show()

# # Plot
# plot_combined_distributions(rent_data, df1)

drop nan on body type

In [8]:
rent_data= df1.dropna(subset=['body type'])

balanced the data

In [9]:
fit_counts = rent_data['fit'].value_counts()
fit_counts 

fit
fit      128856
small     23182
large     22606
Name: count, dtype: int64

In [10]:
target_size_large = fit_counts['large']


small_data_adjusted = rent_data[rent_data['fit'] == 'small'].sample(n=target_size_large, random_state=42, replace=False)
fit_data_adjusted = rent_data[rent_data['fit'] == 'fit'].sample(n=target_size_large, random_state=42, replace=False)
large_data_adjusted = rent_data[rent_data['fit'] == 'large']


balanced_data_adjusted = pd.concat([small_data_adjusted, fit_data_adjusted, large_data_adjusted])


balanced_data_adjusted = balanced_data_adjusted.sample(frac=1, random_state=42).reset_index(drop=True)

Drop useless column

In [11]:
df = balanced_data_adjusted.drop(
    columns=['bust size', 'weight', 'rented for', 'review_text', 'category', 'height', 'age', 'review_date']
)

ADD BMI feature

In [12]:
df['height_m'] = df['height_cm'] / 100
df['BMI'] = df['weight_kg'] / (df['height_m'] ** 2)
df.drop(columns=['height_m'], inplace=True)

add bmi label

In [13]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi <= 23.9:
        return "Normal weight"
    elif 24 <= bmi <= 27.9:
        return "Overweight"
    elif 28 <= bmi <= 30:
        return "Obese Class I"
    elif 30 < bmi <= 35:
        return "Obese Class II"
    else:
        return "Obese Class III"

df['BMI_Range'] = df['BMI'].apply(categorize_bmi)

In [15]:
df.to_csv('./data/cleaned_data.csv', index=False)